In [ ]:
# !pip install handprint

# handprint -a google creds/handprint_google_credentials.json

# fix protobuf
# pip uninstall protobuf
# pip install protobuf==3.20.*

RED PEN REMOVER

In [ ]:
from PIL import Image, ImageEnhance                                                                
import sys
import os

In [ ]:
image_output_dir = "image_output/"
output_dir = "color_correction_output/"
for name in os.listdir(image_output_dir):
    img = Image.open(image_output_dir + name)
    img = img.convert("RGBA")

    pixdata = img.load()

    # Clean the background noise, if color != white, then set to black.

    # REMOVE RED PEN
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            r, g, b, a = pixdata[x, y]
            
            # REMOVE RED PEN
            if (r - g > 20 and
                r - b > 20 and
                r > 200) :
                
                radius = 2
                
                for i in range(2*radius):
                    for j in range(2*radius):
                        pixdata[x + i - radius, y + j - radius] = (255, 255, 255)

    # img.convert("L")

    # SHARPEN
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(10)

    if (not os.path.exists(output_dir)):
        os.makedirs(output_dir)

    img.save(output_dir + name.replace('outfile_', 'colcor_'))

sections

In [1]:
import os
import shutil
import json 
from pprint import pprint

current_path = os.getcwd() 

input_path = current_path + r"/color_correction_output"
output_dir = current_path + r"/handprint_output"
section_output = current_path + r"/section_output"

In [2]:

def move_file(old_path, new_path):
    os.rename(old_path, new_path)
    # os.replace(old_path, new_path)
    shutil.move(old_path, new_path)


for name in os.listdir(input_path):
    path_name = input_path +"/" + name
    if (not os.path.exists()):
        command = f"handprint /o {output_dir} /e /s google {path_name} "
        print(command)
        !{command}
    else:
        print(f"path already exists: {path_name} ")
    
for name in os.listdir(input_path):
    if name.startswith('textoutput'):
        move_file(input_path+"/"+name, output_dir+"/"+name)

handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/colcor_i0_student0_0.png 
╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on 
d:\GitHub\PWS-inscannen/color_correction_output/colcor_i0_student0_0.png
Sending to google and waiting for response ...
Got result from google.
Creating annotated image for google.
Saving all data for google.
Saving extracted text for google.
Creating results grid image: 
handprint_output\colcor_i0_student0_0.handprint-all.png
Done with color_correction_output\colcor_i0_student0_0.png
Do

D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 9324 missing from current font.
  font.set_text(s, 0.0, flags=flags)
D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 9324 missing from current font.
  font.set_text(s, 0, flags=flags)


╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on 
d:\GitHub\PWS-inscannen/color_correction_output/colcor_i119_student59_1.png
Sending to google and waiting for response ...
An error occurred (KeyError): 'full_text_annotation'
handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/colcor_i11_student5_1.png 
╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚════════════════════════════════════════════════════

D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 9324 missing from current font.
  font.set_text(s, 0.0, flags=flags)
D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 9324 missing from current font.
  font.set_text(s, 0, flags=flags)


╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on 
d:\GitHub\PWS-inscannen/color_correction_output/colcor_i143_student71_1.png
Sending to google and waiting for response ...
An error occurred (KeyError): 'full_text_annotation'
handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/colcor_i144_student72_0.png 
╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚══════════════════════════════════════════════════

...

╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on 
d:\GitHub\PWS-inscannen/color_correction_output/colcor_i182_student91_0.png
Sending to google and waiting for response ...
Got result from google.
Creating annotated image for google.
Saving all data for google.
Saving extracted text for google.
Creating results grid image: 
handprint_output\colcor_i182_student91_0.handprint-all.png
Done with color_correction_output\colcor_i182_student91_0.png
Done.
handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/colcor_i183_s

D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 9400 missing from current font.
  font.set_text(s, 0.0, flags=flags)
D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 9400 missing from current font.
  font.set_text(s, 0, flags=flags)


╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on 
d:\GitHub\PWS-inscannen/color_correction_output/colcor_i208_student104_0.png
Sending to google and waiting for response ...
Got result from google.
Creating annotated image for google.
Saving all data for google.
Saving extracted text for google.
Creating results grid image: 
handprint_output\colcor_i208_student104_0.handprint-all.png
Done with color_correction_output\colcor_i208_student104_0.png
Done.
handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/colcor_i20

D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 9324 missing from current font.
  font.set_text(s, 0.0, flags=flags)
D:\GitHub\PWS-inscannen\.venv\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 9324 missing from current font.
  font.set_text(s, 0, flags=flags)


╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on 
d:\GitHub\PWS-inscannen/color_correction_output/colcor_i209_student104_1.png
Sending to google and waiting for response ...
An error occurred (KeyError): 'full_text_annotation'
handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/colcor_i20_student10_0.png 
╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚══════════════════════════════════════════════════

In [12]:
from PIL import Image, ImageDraw
import re

output_items = os.listdir(output_dir)



def filenameify(s):
    replace_dict = {
        "①": "1",
        "②": "2",
        "③": "3",
        "④": "4",
        "⑤": "5",
        "⑥": "6",
        "⑦": "7",
        "⑧": "8",
        "⑨": "9",
        "⑩": "10",
        "⑪": "11",
        "⑫": "12",
        "⑬": "13",
        "⑭": "14",
        "⑮": "15",
        "⑯": "16",
        "⑰": "17",
        "⑱": "18",
        "⑲": "19",
        "⑳": "20",
        ".": ""
    }
    
    # https://www.geeksforgeeks.org/python-replace-words-from-dictionary/
    replaced_s = " ".join(replace_dict.get(ele, ele) for ele in s.split())
    
    filename = re.sub(r'[^a-zA-Z0-9._-]', '', replaced_s)
    
    if (len(filename) == 0):
        return "NOT_VALID_QUESTION_ID"
    
    return filename

def clamp(n, min, max): 
    if n < min: 
        return min
    elif n > max: 
        return max
    else: 
        return n 

for file_name in os.listdir(input_path):
    clean_file_name = file_name.split(".")
    clean_file_name.pop()
    name = ".".join(clean_file_name)
    
    output_json_file_name = name + ".handprint-google.json"
    output_image_file_name = name + ".handprint-google.png"
    output_name = name.replace('colcor_', '')
    if (output_json_file_name not in output_items):
        continue
    
    # output json exists
    with open(output_dir+"/"+output_json_file_name, "r") as f:
        # file_text = f.read()
        text_data = json.load(f)
    
    # print(json.dumps(text_data["text_annotations"][0], sort_keys=True, indent=4))
    horizontal_breaks = []
    for data in text_data["text_annotations"]:
        if (data["bounding_poly"]["vertices"][0]["x"] < 80):
            # if (data["description"].isnumeric()):
            horizontal_breaks.append({"height": data["bounding_poly"]["vertices"][0]["y"], "description": data["description"]})
    print(horizontal_breaks)
    
    image = Image.open(input_path+"/"+file_name)
    output_image = ImageDraw.Draw(image)
    for h_break in horizontal_breaks:
        y = h_break["height"]
        output_image.line([(0, y-15), (image.width, y-15)], fill="red", width=3,)

    main_directory = section_output+"/"+output_name
    try:
        os.makedirs(main_directory)
    except:
        pass
    # image.show()
    image.save(main_directory+"/"+output_name+'.png')
    
    for index, h_break in enumerate(horizontal_breaks):
        if (index == 0 ):
            continue
        previous_y = clamp(horizontal_breaks[index-1]["height"] - 15, 0, image.height - 1)
        y = clamp(h_break["height"] - 15, 0, image.height - 1)
        
        # min height and fix
        if (previous_y > y or y - previous_y < 30):
            continue
        # kantlijn, boven, einde van pagina, beneden grens
        crop = (90, previous_y, image.width, y)
        
        print(crop, image)
        section_image = image.copy()
        cropped = section_image.crop(crop)
        section_directory = main_directory+"/sections"
        try:
            os.makedirs(section_directory)
        except:
            pass
        # section_image.show()
        section_file_name = filenameify(h_break["description"])+".png"
        try:
            cropped.save(section_directory+"/"+section_file_name)
        except:
            print("Could not save: ", section_file_name)
        


[{'height': 118, 'description': 'Opgave\nWhetheme\nexothame Reactive\n- 2 C7 +1165 +1202 >7C02 +8H120 502\n3= de hatalysator bragt de\nGeuver ings exerque Umland\nWandlagen de reactive snelhedel Snellen 15.\nLE\n120\ngeactiveerde toestand\nCHHILSTOR DEGO\n- S:\nbi hele\npindate\nWemale\ngerator\n"Pinda\'s\nReactieproduct\nCo₂ tot 120 + Soz\nVerloop readve'}, {'height': 199, 'description': '-'}, {'height': 198, 'description': '2'}, {'height': 254, 'description': '3'}, {'height': 370, 'description': 'LE'}, {'height': 569, 'description': '-'}, {'height': 572, 'description': 'S'}, {'height': 650, 'description': 'bi'}]
(90, 103, 600, 184) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 at 0x2141E5E7F10>
(90, 183, 600, 239) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 at 0x2141E5E7F10>
(90, 239, 600, 355) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 at 0x2141E5E7F10>
(90, 355, 600, 554) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x